## AI - Research Engine for Commercial Courts

In [27]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_together import Together
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS


from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [28]:
from langchain_community.document_loaders import PyPDFLoader
import streamlit as st

loader = PyPDFLoader("Case-Test.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'Case-Test.pdf', 'page': 0}, page_content="2 All]                     Manoj Kumar Pandey and others V. State of U.P. and another 593\nORIGINAL JURISDICTION \nCIVIL SIDE \nDATED: ALLAHABAD 27.02.2006 \n \nBEFORE \nTHE HON’BLE DR. B.S. CHAUHAN, J. \nTHE HON’BLE DILIP GUPTA, J. \n \nCivil Misc. Writ Petition No. 40736 of 2002 \n \nManoj Kumar Pandey & others ...Petitioners \nVersus \nState of U.P. and another   ...Opp. parties \n \nCounsel for the Petitioners: \nSri S.P. Pandey \nSri D.P. Shukla \n \nCounsel for the Opposite Parties: Sri B.N. Singh \n \nConstitution of India, Art. 226 -Right to \nappointment-petitioner appeared in competative examination-held for the Post of A.P.O. result declared on 20.3.99-State Government send requisition 26.7.01-petition filed in September 2002 e.g. much after expiry of the life of waiting list-parity can not be claimed.  Held: Para 9 & 10  The Hon'ble Apex Court while considering the case has granted relief only to those

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': 'Case-Test.pdf', 'page': 0}, page_content="2 All]                     Manoj Kumar Pandey and others V. State of U.P. and another 593\nORIGINAL JURISDICTION \nCIVIL SIDE \nDATED: ALLAHABAD 27.02.2006 \n \nBEFORE \nTHE HON’BLE DR. B.S. CHAUHAN, J. \nTHE HON’BLE DILIP GUPTA, J. \n \nCivil Misc. Writ Petition No. 40736 of 2002 \n \nManoj Kumar Pandey & others ...Petitioners \nVersus \nState of U.P. and another   ...Opp. parties \n \nCounsel for the Petitioners: \nSri S.P. Pandey \nSri D.P. Shukla \n \nCounsel for the Opposite Parties: Sri B.N. Singh \n \nConstitution of India, Art. 226 -Right to \nappointment-petitioner appeared in competative examination-held for the Post of A.P.O. result declared on 20.3.99-State Government send requisition 26.7.01-petition filed in September 2002 e.g. much after expiry of the life of waiting list-parity can not be claimed.  Held: Para 9 & 10  The Hon'ble Apex Court while considering the case has granted relief only to those

In [30]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'Case-Test.pdf', 'page': 0}, page_content="2 All]                     Manoj Kumar Pandey and others V. State of U.P. and another 593\nORIGINAL JURISDICTION \nCIVIL SIDE \nDATED: ALLAHABAD 27.02.2006 \n \nBEFORE \nTHE HON’BLE DR. B.S. CHAUHAN, J. \nTHE HON’BLE DILIP GUPTA, J. \n \nCivil Misc. Writ Petition No. 40736 of 2002 \n \nManoj Kumar Pandey & others ...Petitioners \nVersus \nState of U.P. and another   ...Opp. parties \n \nCounsel for the Petitioners: \nSri S.P. Pandey \nSri D.P. Shukla \n \nCounsel for the Opposite Parties: Sri B.N. Singh \n \nConstitution of India, Art. 226 -Right to \nappointment-petitioner appeared in competative examination-held for the Post of A.P.O. result declared on 20.3.99-State Government send requisition 26.7.01-petition filed in September 2002 e.g. much after expiry of the life of waiting list-parity can not be claimed.  Held: Para 9 & 10  The Hon'ble Apex Court while considering the case has granted relief only to those

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

def embed_text(texts):
    """
    Takes a list of text strings and returns their embeddings.
    
    Args:
        texts (list): A list of document text strings.
        
    Returns:
        torch.Tensor: A tensor containing the embeddings of the input texts.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
    model = AutoModel.from_pretrained("law-ai/InLegalBERT")
    
    # Tokenize and encode the texts in batches
    encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    
    # Pass the inputs through the model
    with torch.no_grad():  # Disable gradient calculation for efficiency
        model_output = model(**encoded_inputs)
    
    # The embeddings are typically extracted from the last hidden state of the [CLS] token
    embeddings = model_output.last_hidden_state[:, 0, :]  # Taking the [CLS] token embedding for each text
    
    return embeddings


In [34]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:25],OllamaEmbeddings())

In [36]:
db

In [40]:
query="An attention function can be described as mapping a query "
result=db.similarity_search(query)
result[0].page_content

'lacks merit and is accordingly dismissed.  \n---------'

In [39]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama3.1")
llm

Ollama(model='llama3.1')

In [43]:
from langchain_core.prompts import ChatPromptTemplate

# Define a more flexible and clear prompt template
# prompt_template = """
# Answer the following question based strictly on the provided context. 
# Provide a thoughtful, step-by-step explanation before giving your final answer. 
# A bonus of $1000 is yours if the user deems the answer helpful!

# Context:
# {context}

# Question: {input}
# """

# Create the ChatPromptTemplate from the improved template

prompt=ChatPromptTemplate.from_messages (
    [
        ("system", "You are a helpful assistant. Please ask me any questions !!!"),
        ("user", "Context: {context}, Input: {input}")
    ]
)
# prompt = ChatPromptTemplate.from_template(prompt_template)

In [44]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [46]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022D61E07750>)

In [47]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [49]:
response=retrieval_chain.invoke({"input":"Vodafone vs Union of India"})

In [53]:
arr = response['answer'].split("\n\n")

for i in arr:
    print(i)

What a delightful context!
Here are my questions to help me better understand the case:
1. Can you explain why the petitioners' case is distinguishable from others who had approached the court within one year of the last recommendation being made?
2. What specific relief were those other persons granted by the Hon'ble Apex Court, and how does it differ from what the petitioners are seeking?
3. In Srawan Kumar & Ors Vs. Director General of Health Services & Ors, 1993 (Supp) 1 SCC 632, what was the court's holding regarding giving benefits strictly in accordance with merit?
4. You mentioned two cases where the Supreme Court considered granting benefits to a particular person or group despite their belated approach to the court: K.C. Sharma & Ors. Vs. Union of India & Ors., (1997) 6 SCC 721, and S.M. Kotrayya & Ors., (1996) 6 SCC 267. What were the outcomes of these cases?
5. In Jagdish Lal & Ors. Vs. State of Haryana & Ors., AIR 1997 SC 2366, what was the court's observation regarding be